<a href="https://colab.research.google.com/github/vik01/MSCI-436-House-Price-ML-Project/blob/main/MSCI436_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductions

**Imports:**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import io

**Get data from host directory:**

In [ ]:
! rm -r data
! mkdir data
! wget -O houseprices.zip https://kevinjiang.ca/privfiles/houseprices.zip
! unzip houseprices.zip -d data

rm: cannot remove 'data': No such file or directory
--2023-07-12 00:25:58--  https://kevinjiang.ca/privfiles/houseprices.zip
Resolving kevinjiang.ca (kevinjiang.ca)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to kevinjiang.ca (kevinjiang.ca)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203809 (199K) [application/zip]
Saving to: ‘houseprices.zip’

houseprices.zip     100%[===================>] 199.03K  --.-KB/s    in 0.03s   

2023-07-12 00:25:58 (7.53 MB/s) - ‘houseprices.zip’ saved [203809/203809]

Archive:  houseprices.zip
  inflating: data/data_description.txt  
  inflating: data/sample_submission.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


**Load data:**

In [ ]:
test_data = pd.read_csv("data/test.csv")
test_y = pd.read_csv("data/sample_submission.csv")
train_data = pd.read_csv("data/train.csv")

# Add data into dataframes

In [ ]:
cat_to_num = ['MSSubClass', 'OverallQual', 'OverallCond']
numeric_df = train_data.select_dtypes(include = ['int64', 'float64']).drop(columns = 'SalePrice')
numeric_df = numeric_df.select_dtypes(include = ['int64', 'float64']).drop(columns = cat_to_num)
non_numeric_columns = train_data.columns[(train_data.dtypes != 'int64') & (train_data.dtypes != 'float64')].tolist()
non_numeric_columns.extend(cat_to_num)

numeric_df.shape

(1460, 34)

In [ ]:
null_cols = numeric_df.columns[numeric_df.isna().any()].tolist()
# impute NULL values with mean (LotFrontage, MasVnrArea, GarageYrBlt)
for col in null_cols:
  mean = numeric_df[col].mean()
  numeric_df[col].fillna(value = mean, inplace = True)

In [ ]:
from scipy import stats

numeric_df[(np.abs(stats.zscore(numeric_df.drop(columns = 'Id'))) < 3).all(axis=1)]
# numeric_df[np.abs(stats.zscore(numeric_df.drop(columns = 'Id'))) < 3]

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,65.0,8450,2003,2003,196.0,706,0,150,856,...,548,0,61,0,0,0,0,0,2,2008
2,3,68.0,11250,2001,2002,162.0,486,0,434,920,...,608,0,42,0,0,0,0,0,9,2008
4,5,84.0,14260,2000,2000,350.0,655,0,490,1145,...,836,192,84,0,0,0,0,0,12,2008
6,7,75.0,10084,2004,2005,186.0,1369,0,317,1686,...,636,255,57,0,0,0,0,0,8,2007
10,11,70.0,11200,1965,1965,0.0,906,0,134,1040,...,384,0,0,0,0,0,0,0,2,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,1453,35.0,3675,2005,2005,80.0,547,0,0,547,...,525,0,28,0,0,0,0,0,5,2006
1453,1454,90.0,17217,2006,2006,0.0,0,0,1140,1140,...,0,36,56,0,0,0,0,0,7,2006
1454,1455,62.0,7500,2004,2005,0.0,410,0,811,1221,...,400,0,113,0,0,0,0,0,10,2009
1455,1456,62.0,7917,1999,2000,0.0,0,0,953,953,...,460,0,40,0,0,0,0,0,8,2007


In [ ]:
numeric_df['EnclosedPorch'].unique()

array([  0, 272, 228, 205, 176,  87, 172, 102,  37, 144,  64, 114, 202,
       128, 156,  44,  77, 192, 140, 180, 183,  39, 184,  40, 552,  30,
       126,  96,  60, 150, 120, 112, 252,  52, 224, 234, 244, 268, 137,
        24, 108, 294, 177, 218, 242,  91, 160, 130, 169, 105,  34, 248,
       236,  32,  80, 115, 291, 116, 158, 210,  36, 200,  84, 148, 136,
       240,  54, 100, 189, 293, 164, 216, 239,  67,  90,  56, 129,  98,
       143,  70, 386, 154, 185, 134, 196, 264, 275, 230, 254,  68, 194,
       318,  48,  94, 138, 226, 174,  19, 170, 220, 214, 280, 190, 330,
       208, 145, 259,  81,  42, 123, 162, 286, 168,  20, 301, 198, 221,
       212,  50,  99])

In [ ]:
stats.zscore(numeric_df['LotFrontage'])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1455   NaN
1456   NaN
1457   NaN
1458   NaN
1459   NaN
Name: LotFrontage, Length: 1460, dtype: float64

In [ ]:
numeric_df['EnclosedPorch'].describe()

count    1460.000000
mean       21.954110
std        61.119149
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       552.000000
Name: EnclosedPorch, dtype: float64

**Standardize the data**

In [ ]:
# How to 'hide' specific columns from standardization (first answer): https://stackoverflow.com/questions/24645153/pandas-dataframe-columns-scaling-with-sklearn
from sklearn.preprocessing import StandardScaler

standard_transformer_columns = [col for col in numeric_df.columns if col != 'Id']
print(standard_transformer_columns)

['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [ ]:
# scaler = StandardScaler().fit(numeric_df.drop(columns = ['Id']))
scaler = StandardScaler()

In [ ]:
numeric_df[standard_transformer_columns] = scaler.fit_transform(numeric_df[standard_transformer_columns])

In [ ]:
# scaler.mean_
numeric_df.head()

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,-0.208034,-0.207142,1.050994,0.878668,0.510015,0.575425,-0.288653,-0.944591,-0.459303,...,0.351000,-0.752176,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777
1,2,0.409895,-0.091886,0.156734,-0.429577,-0.572835,1.171992,-0.288653,-0.641228,0.466465,...,-0.060731,1.626195,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439
2,3,-0.084449,0.073480,0.984752,0.830215,0.322174,0.092907,-0.288653,-0.301643,-0.313369,...,0.631726,-0.752176,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777
3,4,-0.414011,-0.096897,-1.863632,-0.720298,-0.572835,-0.499274,-0.288653,-0.061670,-0.687324,...,0.790804,-0.752176,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655
4,5,0.574676,0.375148,0.951632,0.733308,1.360826,0.463568,-0.288653,-0.174865,0.199680,...,1.698485,0.780197,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777


In [ ]:
print(f"{non_numeric_columns[0]}", end = "")
for column in non_numeric_columns[1:]:
  print(f", {column}", end = "")

MSZoning, Street, Alley, LotShape, LandContour, Utilities, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, ExterQual, ExterCond, Foundation, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, Heating, HeatingQC, CentralAir, Electrical, KitchenQual, Functional, FireplaceQu, GarageType, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC, Fence, MiscFeature, SaleType, SaleCondition, MSSubClass, OverallQual, OverallCond

# One-hot encoding

In [ ]:
# ONE-HOT ENCODE
# https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
def one_hot(df, col, pre):
  encoded = pd.get_dummies(df[col], prefix=pre)
  for column in encoded:
    encoded = encoded.rename(columns={column: col + "_" + column})
  encoded['Id'] = df['Id']
  return encoded

In [ ]:
for col in non_numeric_columns:
  encoded = one_hot(train_data, col, 'is')
  numeric_df = pd.merge(numeric_df, encoded, on='Id', how='left')

In [ ]:
null_cols = numeric_df.columns[numeric_df.isna().any()].tolist()

In [ ]:
# impute NULL values with mean (LotFrontage, MasVnrArea, GarageYrBlt)
for col in null_cols:
  mean = numeric_df[col].mean()
  numeric_df[col].fillna(value = mean, inplace = True)

In [ ]:
numeric_df.head()

,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,OverallQual_is_10,OverallCond_is_1,OverallCond_is_2,OverallCond_is_3,OverallCond_is_4,OverallCond_is_5,OverallCond_is_6,OverallCond_is_7,OverallCond_is_8,OverallCond_is_9
0,1,-0.208034,-0.207142,1.050994,0.878668,0.510015,0.575425,-0.288653,-0.944591,-0.459303,...,0,0,0,0,0,1,0,0,0,0
1,2,0.409895,-0.091886,0.156734,-0.429577,-0.572835,1.171992,-0.288653,-0.641228,0.466465,...,0,0,0,0,0,0,0,0,1,0
2,3,-0.084449,0.073480,0.984752,0.830215,0.322174,0.092907,-0.288653,-0.301643,-0.313369,...,0,0,0,0,0,1,0,0,0,0
3,4,-0.414011,-0.096897,-1.863632,-0.720298,-0.572835,-0.499274,-0.288653,-0.061670,-0.687324,...,0,0,0,0,0,1,0,0,0,0
4,5,0.574676,0.375148,0.951632,0.733308,1.360826,0.463568,-0.288653,-0.174865,0.199680,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
y_train = train_data["SalePrice"].to_numpy()
x_train = numeric_df.drop(columns = ['Id'])

# Training Model

In [ ]:
reg = LinearRegression().fit(x_train, y_train)
len(reg.coef_)

319

In [ ]:
coeff = pd.DataFrame(list(zip(reg.feature_names_in_, reg.coef_)), columns = ['Feature', 'Weight'])

In [ ]:
coeff.sort_values('Weight')

,Feature,Weight
5,BsmtFinSF1,-2.810650e+16
7,BsmtUnfSF,-2.722953e+16
12,GrLivArea,-1.366895e+16
6,BsmtFinSF2,-9.941108e+15
55,LotConfig_is_FR3,-4.162474e+15
...,...,...
222,KitchenQual_is_Fa,5.464713e+15
221,KitchenQual_is_Ex,5.464713e+15
9,1stFlrSF,1.005603e+16
10,2ndFlrSF,1.135510e+16


#Feature Engineering

In [ ]:
default = ['BldgType_is_1Fam', 'BldgType_is_2fmCon', 'BldgType_is_Duplex', 'BldgType_is_Twnhs', 'BldgType_is_TwnhsE', 'FullBath', 'HalfBath', 'BedroomAbvGr']
#'BldgType_is_1Fam', 'BldgType_is_2fmCon', 'BldgType_is_Duplex', 'BldgType_is_Twnhs', 'BldgType_is_TwnhsE', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'YearBuilt',

In [ ]:
reg2 = LinearRegression().fit(x_train[default], y_train)

In [ ]:
reg2.coef_

array([ 8.97390923e+17,  8.97390923e+17,  8.97390923e+17,  8.97390923e+17,
        8.97390923e+17,  4.64640000e+04,  1.83360000e+04, -7.68000000e+03])

In [ ]:
numeric_df = test_data.select_dtypes(include = ['int64', 'float64'])
numeric_df = numeric_df.select_dtypes(include = ['int64', 'float64']).drop(columns = cat_to_num)
non_numeric_columns = test_data.columns[(test_data.dtypes != 'int64') & (test_data.dtypes != 'float64')].tolist()
non_numeric_columns.extend(cat_to_num)

numeric_df[standard_transformer_columns] = scaler.fit_transform(numeric_df[standard_transformer_columns])
for col in non_numeric_columns:
  encoded = one_hot(test_data, col, 'is')
  numeric_df = pd.merge(numeric_df, encoded, on='Id', how='left')

null_cols = numeric_df.columns[numeric_df.isna().any()].tolist()
# impute NULL values with mean (LotFrontage, MasVnrArea, GarageYrBlt)
for col in null_cols:
  mean = numeric_df[col].mean()
  numeric_df[col].fillna(value = mean, inplace = True)
x_test = numeric_df.drop(columns = ['Id', 'MSSubClass_is_150'])

In [ ]:
not_existing = x_train.columns.difference(x_test.columns)
for col in not_existing:
  x_test[col] = 0

In [ ]:
x_test = x_test[default]

In [ ]:
mse_test = mean_squared_error(test_y.iloc[:, :-1], reg2.predict(x_test))
r2 = r2_score(test_y.iloc[:, :-1], reg2.predict(x_test))
print(mse_test)
print(r2)

34331555146.126114
-193536.1506067203


In [ ]:
#-203873.93971206577 ['YearBuilt', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'FullBath', 'HalfBath', 'BedroomAbvGr']
#-195622.71319481815 ['YearBuilt', 'FullBath', 'HalfBath', 'BedroomAbvGr']
#-193536.1506067203 ['BldgType_is_1Fam', 'BldgType_is_2fmCon', 'BldgType_is_Duplex', 'BldgType_is_Twnhs', 'BldgType_is_TwnhsE', 'FullBath', 'HalfBath', 'BedroomAbvGr']

:# Testing Model

# Dummy Model for development


In [ ]:
test_data_lol = pd.read_csv("data/test.csv")
y_lol = pd.read_csv("data/sample_submission.csv")
train_data_lol = pd.read_csv("data/train.csv")

test_data_lol.head()
train_data_lol.head()
y_lol.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [ ]:
X_meh = train_data_lol[['LotArea', 'BedroomAbvGr', 'YearBuilt', 'FullBath', 'HalfBath']]
y_meh = train_data_lol[['SalePrice']]



In [ ]:

reg = LinearRegression().fit(X_meh, y_meh)
reg.coef_



array([[ 1.68092109e+00, -5.32493704e+02,  8.09952546e+02,
         5.34137866e+04,  2.48171163e+04]])

In [ ]:
# Pickle!
import joblib
with open('bad_model.pkl', 'wb') as fid:
    joblib.dump(reg, fid, compress=9)


In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import joblib

with open('bad_model.pkl', 'rb') as f:
  model = joblib.load(f)

  x = st.slider('Select a value')
  st.write(x, 'squared is', x * x)


Writing app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.075s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.236.193.204


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 2.983s
your url is: https://fluffy-zebras-smash.loca.lt
^C
